In [1]:
from maze_common import *

def run_firstStrat(maze):
    m = maze
    path = None
    result = shortestPathSearch(m)
    if result != None:
        path = result[0]
    #print(path)
    if path == None:
        return False
    # using the initial path generated, iterate through the path, checking if one of the cells along the path catches fire
    for p in path:
        checki = p[0] 
        checkj = p[1]
        #print(checki)
        if maze.board[(checki,checkj)] == 2: # if the current cell is fire, return False
            return False
        elif checki == maze.dim - 1 and checkj == maze.dim - 1: # if goal, return True
            return True
        else:
            m.iterateFire()

ModuleNotFoundError: No module named 'maze_common'

In [ ]:
def run_secondStrat(maze):
    """
    Algorithm:
    1. Identify shortest path to goal
        1.1. If found, move agent according to path
        1.2. Else, keep the agent in place
    2. Check if the agent reached the goal
        2.1. If true, terminate and return success
        2.2. Else, continue
    3. Advance the fire and check if the agent is burned
        3.1. If burned, terminate and return failure
        3.2. Else, continue at step 1
    """
    m = maze
    #m.startFire()
    agent = (0,0)
    shortestPath = None
    while True: #Loop exit conditions: Agent reaches goal or agent gets burned
        result = shortestPathSearch(m, agent) #recalculate the shortest path from the agent to the goal at agent's turn
        if result != None:
            shortestPath = result[0]
        else:
            return False
        if shortestPath != None: #if a shortest path was found, move the agent in that direction
            agent = shortestPath[1]
        if agent == (m.dim-1, m.dim-1): #returns true if the agent reaches the goal, otherwise continue
            return True
        m.iterateFire() #advances the fire immediately after the agent's turn
        if m.board[agent] == 2: #returns false if the fire reaches the agent, otherwise continue
            return False

In [ ]:
def run_thirdStrat(maze):
    """
    Algorithm:
    1. Identify shortest path to goal
        1.1. If found, move agent according to path
        1.2. Else, keep the agent in place
    2. Check if the agent reached the goal
        2.1. If true, terminate and return success
        2.2. Else, continue
    3. Advance the fire and check if the agent is burned
        3.1. If burned, terminate and return failure
        3.2. Else, continue at step 1
    """
    m = maze
    #m.startFire()
    agent = (0,0)
    shortestPath = None
    while True: #Loop exit conditions: Agent reaches goal or agent gets burned
        result = shortestPathSearch(m, agent, heuristicFunction=adjacentFireHeuristic) #recalculate the shortest weighted path from the agent to the goal using the adjacentFireHeuristic
        if result != None:
            shortestPath = result[0]
        else:
            return False
        if shortestPath != None: #if a shortest path exists, advance the agent one step along the path
            agent = shortestPath[1]
        if agent == (m.dim-1, m.dim-1): #if the agent reached the goal, return True
            return True
        m.iterateFire() #advances the fire immediately after the agent's turn
        if m.board[agent] == 2: #if the agent's current cell is now on fire, return False
            return False

In [ ]:
def testStrategy(strategy=run_thirdStrat, dim=30, p=0.3):
#     dim = 10
#     p = 0.3
    q = 0.1
    q_values = []
    avgSuccesses = []
    
    while q < 1: # run strategy for each test value of q (flammability rate)
        run_count = 0
        success_count = 0
        fail_count = 0
        
        testMaze = Maze(dim, p, q)
        while(testMaze.isSolvable() == False):
            testMaze = Maze(dim, p, q)
        maze = copy.deepcopy(testMaze)
        
        while run_count < 100: # generate 30 valid(solvable) mazes
            maze.startFire()
            result = strategy(maze)
            if result == True: # if agent survives, increment number of successes for this given q
                success_count +=1
            elif result == None:
                continue
            else:
                fail_count +=1
            run_count += 1
            maze = copy.deepcopy(testMaze)
        
        successRate = success_count / (success_count + fail_count) # calculate success rate given all of the generated mazes for given q
        q_values.append(q)
        avgSuccesses.append(successRate)
        q += 0.05
    
    q_values_array = np.asarray(q_values)
    avgSuccessRate = np.asarray(avgSuccesses)
    
    plt.figure()
    plt.plot(q_values_array, avgSuccessRate, 'o')
    plt.xlabel("flammability rate")
    plt.ylabel("average success rate")
    plt.title(strategy.__name__)
    plt.show()